#  Преобразование текстов в последовательность индексов токенов. Torchtext.

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/text/stable/
* https://pytorch.org/text/stable/vocab.html
* https://pytorch.org/text/stable/transforms.html

## Задачи для совместного разбора

1\. Рассмотрите основные шаги по преобразованию текста в последовательность индексов токенов.

In [ ]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [ ]:
from nltk import word_tokenize
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
corpus_t = [
    word_tokenize(text.lower())
    for text in corpus
]
corpus_t

In [ ]:
corpus_t

[['студенты',
  'усердно',
  'занимаются',
  'стремясь',
  'получить',
  'знания',
  'и',
  'достичь',
  'успеха'],
 ['студенты',
  'активно',
  'участвуют',
  'в',
  'общественной',
  'жизни',
  'университета']]

In [ ]:
words = set()
words.update(corpus_t[0])
words.update(corpus_t[1])
words = list(words)

In [ ]:
itos = words

In [ ]:
stoi = {v: idx for idx, v in enumerate(itos)}

In [ ]:
itos[0], stoi["занимаются"]

('занимаются', 0)

In [ ]:
corpus_i = [
    [stoi[t] for t in tokens]
    for tokens in corpus_t
]

In [ ]:
corpus_i

[[13, 8, 0, 3, 10, 12, 5, 9, 1], [13, 4, 6, 7, 2, 11, 14]]

In [ ]:
import torch as th

In [ ]:
corpus_i[1].extend((-1, -1)) # padding

In [ ]:
th.tensor(corpus_i)

tensor([[13,  8,  0,  3, 10, 12,  5,  9,  1],
        [13,  4,  6,  7,  2, 11, 14, -1, -1]])

2\. Рассмотрите процесс создания `Vocab` из `torchtext`.

In [ ]:
import torchtext

In [ ]:
torchtext.__version__

'0.15.2+cpu'

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
vocab = build_vocab_from_iterator(
    corpus_t
)

In [ ]:
vocab.lookup_indices(["занимаются", "университета"])

[5, 11]

3\. Примените преобразование `AddToken` из пакета `torchtext`.

In [ ]:
from torchtext.transforms import AddToken

In [ ]:
corpus_i

[[13, 8, 0, 3, 10, 12, 5, 9, 1], [13, 4, 6, 7, 2, 11, 14, -1, -1]]

In [ ]:
add = AddToken(token=999, begin=True)
add(corpus_i)

[[999, 13, 8, 0, 3, 10, 12, 5, 9, 1], [999, 13, 4, 6, 7, 2, 11, 14, -1, -1]]

## Задачи для самостоятельного решения

In [1]:
import pandas as pd
import re
import torch
import nltk
from nltk.tokenize import word_tokenize
from torchtext.vocab import build_vocab_from_iterator
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

<p class="task" id="1"></p>

1\. Опишите класс `Vocab`. При создании объекта `Vocab` в конструктор передается набор текстов, предварительно разбитых на токены. Объект должен позволять:
* по токену получить его уникальный индекс (в случае отсутствия токена в словаре вернуть 1)
* по индексу токена получить сам токен (в случае отсутствия токена в словаре вернуть `<UNK>`)

Первые 4 индекса зарезервированы под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.
    
Создайте `Vocab` на основе списка `corpus` и закодируйте каждый токен в предложениях, используя `Vocab`. Выведите полученный результат на экран.

- [ ] Проверено на семинаре

In [2]:
class Vocab:
    def __init__(self, data):
        self.s2i = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}
        self.i2s =  {v: indx for indx, v in enumerate(self.s2i.values())}

        index = 4

        for text in data:
            words = text.split()
            for word in words:
                word = word.lower()
                if word not in self.s2i:
                    self.s2i[word] = index
                    self.i2s[index] = word

                    index += 1

    def itos(self, idx):
        """Возвращает токен по индексу"""
        return self.i2s.get(idx, '<UNK>')

    def stoi(self, s):
        """Возвращает индекс токена"""
        return self.s2i.get(s, 1)

In [3]:
corpus = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками",
]

In [4]:
vocab = Vocab(corpus)

encoded_corpus = []
for text in corpus:
    tokens = text.lower().split()
    encoded_text = [vocab.stoi(token) for token in tokens]
    encoded_corpus.append(encoded_text)

decoded_corpus = []
for encoded_text in encoded_corpus:
    decoded_tokens = [vocab.itos(index) for index in encoded_text]
    decoded_corpus.append(decoded_tokens)


for i in range(len(corpus)):
    print(f"corpus: {corpus[i]}")
    print(f"encoded_corpus: {encoded_corpus[i]}")
    print(f"decoded_corpus: {decoded_corpus[i]}")
    print("\n")

corpus: Маленький котенок игриво прыгает за шариком
encoded_corpus: [4, 5, 6, 7, 8, 9]
decoded_corpus: ['маленький', 'котенок', 'игриво', 'прыгает', 'за', 'шариком']


corpus: Пушистый котик мурлыкает, лежа на солнышке
encoded_corpus: [10, 11, 12, 13, 14, 15]
decoded_corpus: ['пушистый', 'котик', 'мурлыкает,', 'лежа', 'на', 'солнышке']


corpus: Котенок любопытно нюхает цветы в саду
encoded_corpus: [5, 16, 17, 18, 19, 20]
decoded_corpus: ['котенок', 'любопытно', 'нюхает', 'цветы', 'в', 'саду']


corpus: Котик ловко лазает по дереву, исследуя окружающий мир
encoded_corpus: [11, 21, 22, 23, 24, 25, 26, 27]
decoded_corpus: ['котик', 'ловко', 'лазает', 'по', 'дереву,', 'исследуя', 'окружающий', 'мир']


corpus: Спящий котик мило моргает своими яркими глазками
encoded_corpus: [28, 11, 29, 30, 31, 32, 33]
decoded_corpus: ['спящий', 'котик', 'мило', 'моргает', 'своими', 'яркими', 'глазками']




<p class="task" id="2"></p>

2\. Создайте класс `NewsDataset` на основе данных из файла `news.csv`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости (или новостей, если используются срезы) и метки классов для этих новостей. Для кодирования текстов используйте собственную реализацию `Vocab`. Там, где это возможно, возвращайте результат в виде тензора, а не списка.

Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [5]:
data = pd.read_csv('news.csv')
data.head()

,text,label
0,лукашенко пригрозил литовским танкам белорусск...,0
1,российские компании оказались в опасности из з...,1
2,лукашенко объявил об отмене выборов из за напа...,0
3,«роснефть» стала лидером по объему биржевых пр...,1
4,псковской области дадут десятки миллионов рубл...,1


In [6]:
class NewsDataset:
    def __init__(self, path, vocab):
        self.data = pd.read_csv(path)
        self.vocab = vocab
    def __getitem__(self, index):
        row = self.data.iloc[index]
        texts = row['text']
        labels = row['label']

        if isinstance(texts, str):
            words = texts.split()
            words_indexes_tensor = torch.tensor([self.vocab.stoi(word) for word in words])
            return words_indexes_tensor, labels

        else:
          indexes = []
          for i in range(len(texts)):
                words = texts[i].split()
                words_indexes = [self.vocab.stoi(word) for word in words]
                indexes.append((words_indexes, labels[i]))
          return indexes

In [7]:
vocab = Vocab(data['text'])
newsdataset = NewsDataset('news.csv', vocab)
newsdataset[0], newsdataset[:3]

((tensor([4, 5, 6, 7, 8, 9]), 0),
 [([4, 5, 6, 7, 8, 9], 0),
  ([10, 11, 12, 13, 14, 15, 16, 17, 18], 1),
  ([4, 19, 20, 21, 22, 15, 16, 23, 24, 25], 0)])

<p class="task" id="3"></p>

3\. Реализуйте преобразование `Truncate`, которое обрезает каждый текст в батче до `n` токенов. Создайте версию `NewsDataset` с обрезкой предложений до 5 токенов. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [8]:
class Truncate:
    def __init__(self, n):
        self.n = n

    def __call__(self, text):
        return text[:self.n]

In [9]:
class NewsDataset:
    def __init__(self, path, vocab, transform=None):
        self.data = pd.read_csv(path)
        self.vocab = vocab
        self.transform = transform
    def __getitem__(self, index):
        row = self.data.iloc[index]
        texts = row['text']
        labels = row['label']

        if isinstance(texts, str):
            words = texts.split()
            words_indexes_tensor = torch.tensor([self.vocab.stoi(word) for word in words])
            if self.transform:
                words_indexes_tensor = self.transform(words_indexes_tensor)
            return words_indexes_tensor, labels

        else:
          indexes = []
          for i in range(len(texts)):
                words = texts[i].split()
                words_indexes = [self.vocab.stoi(word) for word in words]
                if self.transform:
                    words_indexes = self.transform(words_indexes)
                indexes.append((words_indexes, labels[i]))
          return indexes

In [10]:
truncate = Truncate(5)
vocab = Vocab(data['text'])
newsdataset = NewsDataset('news.csv', vocab, transform=truncate)
newsdataset[0], newsdataset[:3]

((tensor([4, 5, 6, 7, 8]), 0),
 [([4, 5, 6, 7, 8], 0), ([10, 11, 12, 13, 14], 1), ([4, 19, 20, 21, 22], 0)])

<p class="task" id="4"></p>

4\. Реализуйте преобразование `Pad`, которое расширяет каждый текст в батче до `n` токенов значением `pad_idx`. Создайте версию `NewsDataset` с расширением предложений до 30 токенов. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [11]:
class Pad:
    def __init__(self, n, pad_idx):
        self.n = n
        self.pad_idx = pad_idx

    def __call__(self, text):
        padded_text = text + [self.pad_idx] * (self.n - len(text))
        return padded_text

In [12]:
class NewsDataset:
    def __init__(self, path, vocab, transform=None, pad=None):
        self.data = pd.read_csv(path)
        self.vocab = vocab
        self.transform = transform
        self.pad = pad

    def __getitem__(self, index):
        row = self.data.iloc[index]
        texts = row['text']
        labels = row['label']

        if isinstance(texts, str):
            words = texts.split()
            words_indexes = [self.vocab.stoi(word) for word in words]
            if self.transform:
                words_indexes = self.transform(words_indexes)
            if self.pad:
                words_indexes = self.pad(words_indexes)
            words_indexes_tensor = torch.tensor(words_indexes)
            return words_indexes_tensor, labels

        else:
          indexes = []
          for i in range(len(texts)):
                words = texts[i].split()
                words_indexes = [self.vocab.stoi(word) for word in words]
                if self.transform:
                    words_indexes = self.transform(words_indexes)
                if self.pad:
                    words_indexes = self.pad(words_indexes)
                indexes.append((words_indexes, labels[i]))
          return indexes

In [13]:
pad = Pad(11, 777)
vocab = Vocab(data['text'])
newsdataset = NewsDataset('news.csv', vocab, pad=pad)
newsdataset[0], newsdataset[:3]

((tensor([  4,   5,   6,   7,   8,   9, 777, 777, 777, 777, 777]), 0),
 [([4, 5, 6, 7, 8, 9, 777, 777, 777, 777, 777], 0),
  ([10, 11, 12, 13, 14, 15, 16, 17, 18, 777, 777], 1),
  ([4, 19, 20, 21, 22, 15, 16, 23, 24, 25, 777], 0)])

<p class="task" id="5"></p>

5\. Создайте объект `torchtext.vocab.Vocab` на основе данных из файла `news.csv`. Первые 4 индекса зарезервируйте под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`. Опишите класс `NewsDatasetTorchText`, аналогичный по функционалу классу `NewsDataset`, но использующего реализацию `torchtext.vocab.Vocab`. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [26]:
def tokenizer(text):
    return [token.lower() for token in word_tokenize(text)]

special_tokens = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']
vocab = build_vocab_from_iterator(data['text'].apply(lambda x: tokenizer(x)).tolist(), specials=special_tokens)

In [27]:
class NewsDatasetTorchText:
    def __init__(self, path, vocab, transform=None, pad=None):
        self.data = pd.read_csv(path)
        self.vocab = vocab
        self.transform = transform
        self.pad = pad

    def __getitem__(self, index):
        row = self.data.iloc[index]
        texts = row['text']
        labels = row['label']

        if isinstance(texts, str):
            words = texts.split()
            words_indexes = self.vocab.lookup_indices([word for word in words])

            if self.transform:
                words_indexes = self.transform(words_indexes)
            if self.pad:
                words_indexes = self.pad(words_indexes)
            words_indexes_tensor = torch.tensor(words_indexes)
            return words_indexes_tensor, labels

        else:
          indexes = []
          for i in range(len(texts)):
                words = texts[i].split()
                words_indexes = self.vocab.lookup_indices([word for word in words])
                if self.transform:
                    words_indexes = self.transform(words_indexes)
                if self.pad:
                    words_indexes = self.pad(words_indexes)
                indexes.append((words_indexes, labels[i]))
          return indexes

In [28]:
newsdatasettorchtext = NewsDatasetTorchText('news.csv', vocab)
newsdatasettorchtext[0], newsdatasettorchtext[:3]

((tensor([  31,  203, 3355, 5413,  491, 3095]), 0),
 [([31, 203, 3355, 5413, 491, 3095], 0),
  ([27, 147, 3913, 4, 379, 12, 10, 2422, 4484], 1),
  ([31, 63, 29, 4036, 318, 12, 10, 3667, 568, 991], 0)])

<p class="task" id="6"></p>

6\. Создайте преобразование, которое последовательно:
* преобразует набор токенов в последовательность индексов;
* преобразует результат в тензор;
* расширяет предложения до 30 символов, заполняя недостающие позиции индексом 0.

Создайте версию NewsDatasetTorchText с указанием этого преобразования. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [30]:
class NewsDatasetTorchText:
    def __init__(self, path, vocab, transform=None, pad=None):
        self.data = pd.read_csv(path)
        self.vocab = vocab
        self.transform = transform
        self.pad = pad

    def __getitem__(self, index):
        row = self.data.iloc[index]
        texts = row['text']
        labels = row['label']

        if isinstance(texts, str):
            words = texts.split()
            words_indexes = self.vocab.lookup_indices([word for word in words])

            if self.transform:
                words_indexes = self.transform(words_indexes)
            if self.pad:
                words_indexes = self.pad(words_indexes)
            words_indexes_tensor = torch.tensor(words_indexes)
            return words_indexes_tensor, labels

        else:
          indexes = []
          for i in range(len(texts)):
                words = texts[i].split()
                words_indexes = self.vocab.lookup_indices([word for word in words])
                if self.transform:
                    words_indexes = self.transform(words_indexes)
                if self.pad:
                    words_indexes = self.pad(words_indexes)
                indexes.append((words_indexes, labels[i]))
          return indexes

In [ ]:
class Transform:
    def __init__(self, n, vocab):
        self.n = n
        self.vocab = vocab
    def __call__(self, tokens):
        token_indices = self.vocab.lookup_indices([token for token in tokens])
        token_tensor = torch.tensor(token_indices)
        token_tensor = torch.cat([token_tensor, torch.zeros(self.n - len(token_indices), dtype=torch.long)])
        return token_tensor

## Обратная связь
- [ ] Хочу получить обратную связь по решению